# 🤗Transformers로 파인튜닝 한 모델 Torchserve로 배포하기

### 다루는 내용

이 글은 🤗Transformers 모델을 학습한 뒤 Torchserve로 배포하는 방법에 대해 소개합니다.


### 🤗Transformers를 활용해 Textclassification 모델 학습하기

- 🤗Transformers에 익숙하지 않은 경우 Huggingface의 기본 기능을 소개하고 구현하는 방법을 다루는 ~~와 ~~를 참고
- 이번에는 Distil-bert 모델을 Textclassification로 Fine-tuning 하겠음. 데이터 선정, 모델 학습은 Huggingface에서 Tutorial을 활용했음.
- 추가적인 내용 이해가 필요한 경우 [Huggingfae Fine-tuning turorial](https://huggingface.co/docs/transformers/training) 예제를 함께 참고하면 좋음


### 학습 & 평가 데이터 만들기

- huggingface에서 제공하는 `datasets`라이브러리를 활용해 학습, 평가 데이터를 생산

- dataload_datset을 활용하면 [Huggingface Datasets](https://huggingface.co/datasets)을 쉽게 불러올 수 있습니다.


In [1]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")


Found cached dataset yelp_review_full (/Users/yangwoolee/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
dataset


DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

내부 데이터 확인을 위해서 train를 선택하고 3개의 데이터만 추출하겠습니다.


In [44]:
dataset["train"][:3]


{'label': [4, 1, 3],
 'text': ["dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.",
  "Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't get it.  You have o

이번 예제에서는 65000개의 학습 데이터 중 1000개의 데이터를 임의로 추출해 학습에 활용하겠습니다.
평가용 데이터는 100개를 추출하겠습니다.


In [19]:
train_dataset = train_dataset = dataset["train"].shuffle(seed=42).select(range(1000))
validation_dataset = dataset["test"].shuffle(seed=42).select(range(100))


Loading cached shuffled indices for dataset at /Users/yangwoolee/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-a0e621c27d9b360e.arrow
Loading cached shuffled indices for dataset at /Users/yangwoolee/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-61e0da4d9cd46a2c.arrow


데이터 추출 과정을 반복하지 않기 위해 csv 파일로 저장하겠습니다. csv로 저장하기 위해 .to_csv를 활용합니다. dataset 내부는 pandas를 활용하므로 pandas의 .to_csv 매서드와 동일한 인자를 활용 할 수 있습니다. 예제에서 설정한 index=False는 csv에 포함된 index를 저장하지 않는 기능입니다.


In [5]:
train_dataset.to_csv("data/train.csv", index=False)
validation_dataset.to_csv("data/validation.csv", index=False)


Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

36382

> 저장한 모델을 불러오는 방법은 다음과 같습니다.


In [41]:
data = load_dataset("csv", data_files={"train": "data/train.csv", "validation": "data/test.csv"})
data


Using custom data configuration default-1f49b8a4a5164bfe
Found cached dataset csv (/Users/yangwoolee/.cache/huggingface/datasets/csv/default-1f49b8a4a5164bfe/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 50
    })
})

### Text classification 모델 Fine-tuning

- 🤗Transformers를 활용하면 Task에 필요한 모델 구조를 쉽게 구현할 수 있음.

- 🤗Transformers는 BaseModel을 기반으로 하되 output-Layer 구조를 변경하는 방법으로 모델 구조를 변경함.

- 구현하고자 하는 모델은 리뷰를 통해 평점을 예측하는 모델이므로 리뷰를 input 데이터로 넣으면 평점(1~5)을 반환하는 구조여야함.

- 이러한 유형의 Task를 Text-classification이라 하며 🤗Transformers의 sequenceClassification 모델을 통해 구현할 수 있음.

- 학습에 활용한 BaseModel은 Distil_bert를 활용했음.


In [2]:
from transformers import (
    DistilBertForSequenceClassification,
    DistilBertTokenizer,
    Trainer,
    TrainingArguments,
    TrainerCallback,
)
from datasets import load_dataset
import pandas as pd

# load data
data = load_dataset("csv", data_files={"train": "data/train.csv", "validation": "data/test.csv"})

train_dataset = data["train"]
evaluation_dataset = data["validation"]


Using custom data configuration default-1f49b8a4a5164bfe
Found cached dataset csv (/Users/yangwoolee/.cache/huggingface/datasets/csv/default-1f49b8a4a5164bfe/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
### 데이터 토크나이징

* 모든 데이터에 대해 토크나이징을 수행하겠습니다.

In [3]:
# tokenizing
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")


def tokenize_function(item):
    return tokenizer(item["text"], padding="max_length", max_length=128, truncation=True)


train = train_dataset.map(tokenize_function)
evaluation = evaluation_dataset.map(tokenize_function)

print("complete Tokenizing")


  0%|          | 0/1000 [00:00<?, ?ex/s]

  0%|          | 0/50 [00:00<?, ?ex/s]

complete Tokenizing


### 🤗Transformers 모델 구조 이해하기

- sequenceclassifcation 구조는 여러 종류의 task에 활용될 수 있다. Regression 모델을 구성하거나 classification 모델을 구성할 때 주로 활용된다.

- input data 구조와, sequenceclassifcation의 인자인 num_label을 변경하면 sequenceclassifcation 구조 하나로 다양한 Task 수행이 가능하다.

- sequenceclassifcation 구조를 활용해 구현 가능한 Task는 Text_classification, SentenceSimilarity, Q&A, Inference 등이 있다. 사실 이 외에도 output으로 regression 또는 classfication 이 필요한 경우 모두 사용가능하다.

- input data 구조는 문장(또는 문단)을 하나만 사용하는 구조와 문장(또는 문단을) 두 개 사용해 비교하는 구조가 있다. Text classification의 경우 전자를 SentenceSimilarity, Q&A, Inference은 후자의 구조를 필요로한다.

- num_label은 output 유형을 결정한다. num_label = 1로 설정하면 0~1사이 범위의 Regression 모델로 활용 가능하고 num_label을 2 이상으로 설정하면 classification 모델로 활용가능하다.

- 🤗Transformers는 sequenceClassification외에도 MaskedLM 구조, TokenClassification 구조 QuestionAnswering 구조 등 다양한 구조를 제공한다.

> Q&A 유형은 세부적으로 두 가지가 있다. 하나는 Question에 대한 답을 생성하는 유형, 다른 하나는 Question에 대한 적절한 답을 보기에서 선택하는 유형이다. 쉽게 말해 주관식, 객관식 문제를 해결하는 방법이 있다고 보면된다. QuestionAnswering은 주관식으로 output을 생성하는 Task이고 Sequeenceclassification은 객관식으로 주어진 답변을 선택하는 Task이다.


### Sequenceclassification Model 학습하기

- 🤗Transformers의 모델 유형을 이해했으니 text-classification 모델을 불러오고 학습하는 방법에 대해 소개하겠다.

- 🤗Transformers의 Trainer API에 대한 소개는 ~~~에서 설명하고 있습니다.


In [4]:
tra_arg = TrainingArguments(
    num_train_epochs=1,
    output_dir="test",
    logging_steps=10,
    # evaluation_strategy="epoch",
)


class myCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        print(f"{state.global_step}회 진행 중 ")


model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=5)


trainer = Trainer(
    model=model, args=tra_arg, train_dataset=train, eval_dataset=evaluation, callbacks=[myCallback]
)

trainer.train()


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier

  0%|          | 0/125 [00:00<?, ?it/s]

10회 진행 중 
{'loss': 1.6229, 'learning_rate': 4.600000000000001e-05, 'epoch': 0.08}
20회 진행 중 
{'loss': 1.5871, 'learning_rate': 4.2e-05, 'epoch': 0.16}
30회 진행 중 
{'loss': 1.5688, 'learning_rate': 3.8e-05, 'epoch': 0.24}
40회 진행 중 
{'loss': 1.5909, 'learning_rate': 3.4000000000000007e-05, 'epoch': 0.32}
50회 진행 중 
{'loss': 1.4308, 'learning_rate': 3e-05, 'epoch': 0.4}
60회 진행 중 
{'loss': 1.3809, 'learning_rate': 2.6000000000000002e-05, 'epoch': 0.48}
70회 진행 중 
{'loss': 1.3381, 'learning_rate': 2.2000000000000003e-05, 'epoch': 0.56}
80회 진행 중 
{'loss': 1.1999, 'learning_rate': 1.8e-05, 'epoch': 0.64}
90회 진행 중 
{'loss': 1.2068, 'learning_rate': 1.4000000000000001e-05, 'epoch': 0.72}
100회 진행 중 
{'loss': 1.2225, 'learning_rate': 1e-05, 'epoch': 0.8}
110회 진행 중 
{'loss': 1.1651, 'learning_rate': 6e-06, 'epoch': 0.88}
120회 진행 중 
{'loss': 1.1877, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.96}




Training completed. Do not forget to share your model on huggingface.co/models =)




125회 진행 중 
{'train_runtime': 95.6653, 'train_samples_per_second': 10.453, 'train_steps_per_second': 1.307, 'train_loss': 1.3701940231323242, 'epoch': 1.0}


TrainOutput(global_step=125, training_loss=1.3701940231323242, metrics={'train_runtime': 95.6653, 'train_samples_per_second': 10.453, 'train_steps_per_second': 1.307, 'train_loss': 1.3701940231323242, 'epoch': 1.0})

### torchserve 생성하기

- Huggingface는 pytorch를 기반으로 구성됐기 때문에 torchserve로 배포하는 것이 가능합니다.

- Huggingface 모델을 torchserve로 배포하는 방법은 다음과 같습니다.

- 모델, tokenizer 저장, 핸들러(Handler) 제작 => MAR file 생성 => torchserve로 배포

- MAR file은 ~~~를 통해 만들 수 있고 이를 위해선 Handler와 model, tokenzier가 필요합니다.

- Handler 설명에 앞서 trainer를 통해 학습한 모델을 저장하고 토크나이저도 저장하겠습니다. 모두 같은 경로에 저장해주세요.


In [5]:
trainer.save_model("torch_model")
tokenizer.save_pretrained("torch_model")


Saving model checkpoint to torch_model
Configuration saved in torch_model/config.json
Model weights saved in torch_model/pytorch_model.bin
tokenizer config file saved in torch_model/tokenizer_config.json
Special tokens file saved in torch_model/special_tokens_map.json


('torch_model/tokenizer_config.json',
 'torch_model/special_tokens_map.json',
 'torch_model/vocab.txt',
 'torch_model/added_tokens.json')

### Handler가 필요한 이유

- 우리는 지금 전처리된 데이터 셋을 활용해 모델을 학습했습니다. 이러한 방식은 Product 환경에서는 어림도 없는 이야기입니다. Input Data로 수많은 html tag로 뒤덮힌 값이 들어올 수도 있고 Encoding 된 값으로 들어와 decoding해서 사용해야할 수도 있습니다.

- 데이터 전처리만 신경쓸게 아닙니다. 모델 output을 반환할 때 특정 양식에 맞춰서 반환해야한다던가 Metric을 생성한다던가 하는 데이터 후처리도 신경써야합니다.

- 이러한 데이터 전처리 - 추론 - 후처리의 파이프라인을 구성하는데 필요한 기능을 torchserve에서는 Handler라 부릅니다.


### 🤗Transformers를 위한 Handler

- Handler는 BaseHandler Class를 상속받아 작성합니다.

- nn.Module을 사용해서 모델을 제작할 때 forward를 재작성하는 것처럼, Handler 또한 기본적으로 BaseHandler를 불러온 뒤 preprocess, postprocess를 구성하면 됩니다.

- 이때 🤗Transformers를 사용할 시 transformers 라이브러리를 활용해 모델과 토크나이저를 불러와야 하므로 모델을 불러오는 부분인 initialize 함수도 일부 수정해야합니다. 이때 변경할 사항은 두 가지입니다. 하나는 `self.model`, 다른 하나는 `self.tokenizer`입니다. 이 외에는 BaseHandler의 구조와 동일합니다.

- 아래 Handlers는 기본 구조이므로 preprocess와 postprocess의 내용을 입맛에 맞게 변경하시면 됩니다.

  > 주의! Torchserve의 input data는 bytearray로 변환되어 들어옵니다.
  >
  > Input data로 "Stopped back by Mellow Mushroom with my mate Justin from Brew Bros."의 string data를 요청하면 모델이 받는 Input data 구조는 `[{'body': bytearray(b'"Stopped back by Mellow Mushroom with my mate Justin from Brew Bros."')}]` 가 됩니다. 따라서 bytearray를 decode할 수 있게 `txt.decode('utf-8')`코드를 포함해서 작성해야 합니다.

- 끝으로 Handler를 작동시키는 Handle 함수를 생성합니다.


In [ ]:
from abc import ABC
import json
import logging
import os

import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from ts.torch_handler.base_handler import BaseHandler

logger = logging.getLogger(__name__)


class TransformersClassifierHandler(BaseHandler, ABC):
    def __init__(self):
        super(TransformersClassifierHandler, self).__init__()
        self.initialized = False

    def initialize(self, ctx):
        self.manifest = ctx.manifest

        properties = ctx.system_properties
        model_dir = properties.get("model_dir")
        self.device = torch.device(
            "cuda:" + str(properties.get("gpu_id")) if torch.cuda.is_available() else "cpu"
        )

        # Read model serialize/pt file
        self.model = AutoModelForSequenceClassification.from_pretrained(model_dir)
        self.tokenizer = AutoTokenizer.from_pretrained(model_dir)

        self.model.to(self.device)
        self.model.eval()

        logger.debug("Transformer model from path {0} loaded successfully".format(model_dir))

        # Read the mapping file, index to object name
        mapping_file_path = os.path.join(model_dir, "index_to_name.json")

        if os.path.isfile(mapping_file_path):
            with open(mapping_file_path) as f:
                self.mapping = json.load(f)
        else:
            logger.warning(
                "Missing the index_to_name.json file. Inference output will not include class name."
            )

        self.initialized = True

    def preprocess(self, data):
        """Very basic preprocessing code - only tokenizes.
        Extend with your own preprocessing steps as needed.
        """
        print("------- input data 확인 --------")
        print(data)
        text = data[0].get("data")
        if text is None:
            text = data[0].get("body")

        # inpudata로 bytearray를 받으므로 deoce필요
        sentences = text.decode("utf-8")

        logger.info("Received text: '%s'", sentences)

        inputs = self.tokenizer.encode_plus(sentences, add_special_tokens=True, return_tensors="pt")
        return inputs

    def inference(self, inputs):
        """
        Predict the class of a text using a trained transformer model.
        """
        # NOTE: This makes the assumption that your model expects text to be tokenized
        # with "input_ids" and "token_type_ids" - which is true for some popular transformer models, e.g. bert.
        # If your transformer model expects different tokenization, adapt this code to suit
        # its expected input format.
        inputs = inputs.to(self.device)

        prediction = self.model(**inputs)[0].argmax().item()
        logger.info("Model predicted: '%s'", prediction)

        if self.mapping:
            prediction = self.mapping[str(prediction)]
        return [prediction]

    def postprocess(self, inference_output):
        # TODO: Add any needed post-processing of the model predictions here
        logger.info("Model Name: '%s'", self.model.config._name_or_path)
        logger.info("Model predicted: '%s'", inference_output)
        return inference_output


_service = TransformersClassifierHandler()


def handle(data, context):
    try:
        if not _service.initialized:
            _service.initialize(context)

        if data is None:
            return None

        data = _service.preprocess(data)
        data = _service.inference(data)
        data = _service.postprocess(data)

        return data
    except Exception as e:
        raise e


### MAR file 생성하기

- 이제 Mar file을 생성할 수 있는 조건이 갖춰졌습니다. model, tokenizer, handler가 동일한 경로에 있는지 다시 한 번 확인해주세요.

- Mar 생성은 커맨드에서 `torch-model-archiver`를 실행해 생성합니다.

- Terminal을 켜서 model, tokenizer, handler가 있는 경로로 이동합니다. 아래의 명령어를 본인이 저장한 경로에 맞게 수정해주세요.
  `--serialized-file=pytorch_model.bin`, `--handler "./torch_serve_for_kserve.py"`, `--extra-files "./bert_model/config.json,./bert_model/vocab.txt"` 이 부분을 수정해주세요.

  > Torchserve 활용을 위해 `pip install torchserve torch-model-archiver torch-workflow-archiver`을 우선 설치해주세요

```bash
torch-model-archiver --model-name bert-model --version 1.0 --serialized-file ./bert_model/pytorch_model.bin  --handler "./torch_serve_for_kserve.py" --extra-files "./bert_model/config.json,./bert_model/vocab.txt"
```

- 그럼 해당 경로에 bert-model.mar 이라는 파일이 새로 생긴것을 확인할 수 있습니다.

- mar File의 내부구조는 args에 포함한 파일 + MAR_INF 폴더 내부에 있는 json 파일로 구성되어 있습니다.

  ```json
  # MAR_INF 내부 Json 정보
  {
    "createdOn": "17/01/2023 18:36:16",
    "runtime": "python",
    "model": {
      "modelName": "bert-model",
      "serializedFile": "pytorch_model.bin",
      "handler": "torch_serve_for_kserve.py",
      "modelVersion": "1.0"
    },
    "archiverVersion": "0.7.0"
  }
  ```


### TorchServe 배포하기

- 이제 모델을 배포할 일만 남았습니다. 우선 model_store 폴더를 만든 뒤 bert-model.mar 파일을 내부로 이동해주세요.

- 다음의 명령어를 샐행해 정상 작동하는지 확인해보겠습니다.

- `--model-store`는 폴더 경로, `--models`은 `모델명`과 MarFile을 설정합니다.

- `모델명`은 API Endpoint로 활용되므로 기억해주셔야합니다.

  ```bash

  torchserve --start --model-store model_store --models bert=bert_model.mar

  ```

> Error log 중 empty snapshot으로 뜨는 경우 `--no-config-snapshots`을 추가로 포함해주세요.

- 이제 새로운 Terminal을 띄우고 API에 접근해보겠습니다.

- torchserve에서 사용했던 `모델명`을 활용해 주소를 http://127.0.0.1:8080/predictions/`모델명`으로 변경해야합니다.

```bash

curl -X POST http://127.0.0.1:8080/predictions/bert -d Stopped back by Mellow Mushroom with my mate Justin from Brew Bros.

결과 :
```

- Torchserve가 켜진 환경을 보면 Input data가 정확히 들어왔음을 확인할 수 있습니다.


In [ ]:
### TorchServe를 컨테이너로 제작하기

* 이번엔 Torchserve를 통해 배포한 모델을 컨테이너로 제작하겠습니다.


### KFP를 활용해 Training PipeLine 만들기


In [1]:
#### New
from functools import partial
from kfp.components import create_component_from_func, InputPath, OutputPath


@partial(
    create_component_from_func,
    packages_to_install=["pandas"],
)
def load_data(
    train_path: OutputPath("csv"),
    evaluation_path: OutputPath("csv"),
):

    import pandas as pd

    # load data from github
    df_train = pd.read_csv(
        "https://raw.github.com/yangoos57/Learning_kubeflow/main/mini_project/data/train.csv"
    )
    df_evaluation = pd.read_csv(
        "https://raw.github.com/yangoos57/Learning_kubeflow/main/mini_project/data/validation.csv"
    )

    df_train.to_csv(train_path, index=False)
    df_evaluation.to_csv(evaluation_path, index=False)

    print("complete Loading Data")


In [4]:
@partial(create_component_from_func, base_image="679oose/basepython:1.0")
def train_model(
    train_path: InputPath("csv"),
    evaluation_path: InputPath("csv"),
    model_save_path: OutputPath("folder"),
):

    from transformers import (
        DistilBertForSequenceClassification,
        DistilBertTokenizer,
        Trainer,
        TrainingArguments,
        TrainerCallback,
    )
    from datasets import Dataset

    # loading data
    train_dataset = Dataset.from_csv(train_path)
    evaluation_dataset = Dataset.from_csv(evaluation_path)

    # tokenizing
    tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

    def tokenize_function(item):
        return tokenizer(item["text"], padding="max_length", max_length=128, truncation=True)

    train = train_dataset.map(tokenize_function)
    evaluation = evaluation_dataset.map(tokenize_function)

    print("complete Tokenizing")

    model = DistilBertForSequenceClassification.from_pretrained(
        "distilbert-base-uncased", num_labels=len(set(train_dataset["label"]))
    )
    tra_arg = TrainingArguments(
        output_dir="test",
        num_train_epochs=1,
        logging_steps=5,
        evaluation_strategy="epoch",
        disable_tqdm=True,
        save_strategy="no",
    )

    class myCallback(TrainerCallback):
        def on_log(self, args, state, control, logs=None, **kwargs):
            print(f"{state.global_step} Steps ")

    trainer = Trainer(
        model=model,
        args=tra_arg,
        train_dataset=train,
        eval_dataset=evaluation,
        callbacks=[myCallback],
    )

    trainer.train()
    trainer.save_model(model_save_path)


In [6]:
from kfp.dsl import pipeline


@pipeline(name="NLP_Pipeline")
def NLP_Pipeline():
    data = load_data()
    train_model(data.outputs["train"], data.outputs["evaluation"])


import kfp

if __name__ == "__main__":
    kfp.compiler.Compiler().compile(NLP_Pipeline, "NLP_Pipeline_1.2.yaml")
